In [135]:
import pickle
import pandas as pd
with open('/home3/data/LUNG_CANCER/sql/NLP_data/cleaned_data/radiology_report/radiology_report_use_finial.pickle') as data:
     radiology_report_use_finial=pickle.load(data)


In [1]:
import pickle
def save_file(folder_file_name,var):
    with open(folder_file_name, 'wb') as file2save:
        pickle.dump(var, file2save)
        
def load_file(file_name):
    with open(file_name, 'rb') as file2load:
        return pickle.load(file2load)
    

In [137]:
text_segmented_raw=load_file('/home3/data/LUNG_CANCER/sql/NLP_data/cleaned_data/radiology_report/text_segmented_raw')

# merge with brain metastasis

In [138]:
BrainMets_MRN=pd.read_csv('./BrainMets_MRN.csv')

In [151]:
# BrainMets_MRN.head()
# BrainMets_MRN.columns
BrainMets_MRN['MRN'].dtype,

dtype('int64')

In [176]:
# radiology_report_use_finial.loc[:,'MRN']=radiology_report_use_finial['PatNum']
# radiology_report_use_finial.columns
# radiology_report_use_finial.head()
# radiology_report_use_finial['MRN'].dtype
radiology_report_use_finial['RenderedText'][0]

'IMPRESSION: _ 1. Probable combination of homogeneous infiltrates and pleural effusions in the mid and lower lung fields.  The right side is increased in density from the prior day and the left side is decreased.  The infiltrates were more extensive involving the upper lobes on December 30, 2008.2. No pneumothorax  HISTORY: Intubated.  AML.Exam Date 01-01-2009          ACC# 867571FULL RESULT: A portable chest x-ray was obtained on January 1, 2009, 6:29 a.m.. Homogeneous infiltrates and probable pleural effusions in the lower lobes are again seen.  The density in the right lower lobe is increased from the prior day.  The left lower lobe density is possibly decreased.  No pneumothorax is seen.  The heart is possibly mildly enlarged.  The mediastinum is midline. The endotracheal tube, NG tube and central venous catheter are in good position. Electronically Approved By ARNOLD ROTTER, /PSC   TECH: IJCdd: 01-01-2009 15:07ds: 01-01-2009 15:11STATUS: FINAL'

In [157]:
# merge
# we use ResultSrcNumID as key to connect patient information and text
# TestItemDescr tell us what kind of report it is
# radiology_report_use_finial.loc[:,['MRN','TestItemDescr','ResultDatm','ResultSrcNumID','RenderedText']]
BrainMets_MRN_report=pd.merge(BrainMets_MRN,radiology_report_use_finial.loc[:,['MRN','TestItemDescr','ResultDatm','ResultSrcNumID','RenderedText']],on='MRN')
BrainMets_MRN_report.head()

,MRN,Resourse,TestItemDescr,ResultDatm,ResultSrcNumID,RenderedText
0,10060252,Allscripts,XR Chest AP Portable,2013-09-12 09:17:00.000,113695835,IMPRESSION: Bibasilar pleural-parenchymal chan...
1,10180581,Allscripts,CT Thoracic Spine w Contrast,2017-07-24 10:03:00.000,426753623,IMPRESSION: 1. Status post prior T6 vertebrec...
2,10180581,Allscripts,XR Chest 1 View - (Outside Imaging),2016-01-12 00:00:00.000,430854238,IMPRESSION:Endotracheal tube overlying 5 cm ab...
3,10180581,Allscripts,MRI Thoracic Spine - (Outside Imaging),2016-05-22 00:00:00.000,430934668,IMPRESSION: Sequela posterior spinal decompres...
4,997665,Allscripts,MRI Brain wwo Contrast,2014-12-29 16:22:00.000,217200406,IMPRESSION: 1. Bilateral occipital lobe metast...


In [164]:
# choose brain MRI
BrainMets_MRN_report_BrainMRI=BrainMets_MRN_report[map(lambda x: 'MRI' in x and any(i in x for i in ['Brain','Head']),BrainMets_MRN_report['TestItemDescr'])]
BrainMets_MRN_report_BrainMRI.head()

,MRN,Resourse,TestItemDescr,ResultDatm,ResultSrcNumID,RenderedText
4,997665,Allscripts,MRI Brain wwo Contrast,2014-12-29 16:22:00.000,217200406,IMPRESSION: 1. Bilateral occipital lobe metast...
5,997665,Allscripts,MRI Head without + with Contrast,2012-05-24 12:41:00.000,56386323,IMPRESSION:1. Normal MRI of the brain.. HISTO...
6,997665,Allscripts,MRI Brain/Cervical Spine without + with Contrast,2012-08-09 18:56:00.000,59724202,IMPRESSION:1.Normal brain2. No meningeal metas...
9,997665,Allscripts,MRI Brain wwo Contrast - Rad Onc,2015-01-13 14:25:00.000,221544151,IMPRESSION: 1. Bilateral deep occipital lobe...
10,997665,Allscripts,MRI Brain wwo Contrast - Rad Onc,2015-03-24 13:14:00.000,239590878,IMPRESSION: 1. Compared to prior MRI brain Jan...


In [169]:
BrainMets_MRN_report_BrainMRI_TextSegmented={}
for i in BrainMets_MRN_report_BrainMRI['ResultSrcNumID']:
    BrainMets_MRN_report_BrainMRI_TextSegmented.setdefault(i,text_segmented_raw[i])

In [4]:
BrainMets_MRN_report_BrainMRI_TextSegmented=BrainMets_MRN_use_dict['segmented_report']
BrainMets_MRN_report_BrainMRI=BrainMets_MRN_use_dict['dataframe']

# Report selection

In [5]:
need_check_1=[key for key in BrainMets_MRN_report_BrainMRI_TextSegmented.keys() for key_1 in BrainMets_MRN_report_BrainMRI_TextSegmented[key].keys() if key_1=='need_check']
len(need_check_1)

72

In [173]:
for ID in need_check_1:
            #cp_list.append('/home3/data/LUNG_CANCER/sql/sql_new/radiology_report/'+COH_ID+'/'+ID_time)
        print '='*100
        print '='*100
        print BrainMets_MRN_report_BrainMRI_TextSegmented[ID]

{'need_check': {'weird_patterns': {'HISTORY': [216, 230]}, 'text': ' IMPRESSION :     1. Increased T2 prolongation within posterior parasagittal parietal and within occipital lobes, greater on the left, with a gyriform pattern.  Interval development of restricted diffusion within posterior parietal and occipitVentricles:  Unremarkable.  No hydrocephalus.  Sinuses:  Extensive mucosal reaction within paranasal sinuses, especially maxillary and sphenoid sinuses where there is suggestion of bony expansion and possible mucoceles or possible presence of extra medullary hematopoiesis.  Electronically_Approved  By BRIAN BURGOYNE, M.D./NHAWK TECH: VRMdd: 11-27-2014 17:00ds: 11-28-2014 09:26STATUS: FINALal cortex bilaterally, greater on the left.  Corresponding slight decrease in ADC.  Mild patchy cortical enhancement within regions of T2 prolongation, predominantly in the posterior right parietal lobe and to lesser extent within the left precuneus and left occipital lobe.  Question changes of p

{'ADDENDUM': {'ADDENDUM': 'ADDENDUM  10/06/2017 11:21AM ***Also noted with the given history of right ear pain: The patient has a small heterogeneously enhancing lesion posterior to the right mandibular condyle extending superiorly but not definitely into cephalus.Nonspecific prominent tentorial vein abutting the inferior aspect of the distal cisternal segment of the left trigeminal nerve proximal to the porus acusticus. The left anterior inferior cerebellar artery abuts the inferior aspect of the mid to distal cisternal segment of the left trigeminal on the right side, the right superior cerebellar artery abuts the proximal to mid cisternal segment of the right trigeminal nerve.There is no acute infarct. Stable left parasagittal retrocerebellar arachnoid cyst. There is no new extra-axial fluid collection. Basal cisterns are patent. Orbits, sella and craniocervical junction are stable. Sequela bilateral lens extraction. Mild paranasal sinus mucosal thickening without air-fluid levels. 

In [6]:
 need_check_addendum=[key for key in BrainMets_MRN_report_BrainMRI_TextSegmented.keys() for key_1 in BrainMets_MRN_report_BrainMRI_TextSegmented[key].keys() if key_1=='ADDENDUM']
# len(need_check_addendum)
# '430687809' in need_check_addendum[0:10]

In [174]:
for ID in need_check_addendum:
            #cp_list.append('/home3/data/LUNG_CANCER/sql/sql_new/radiology_report/'+COH_ID+'/'+ID_time)
        print '='*100
        print '='*100
        print BrainMets_MRN_report_BrainMRI_TextSegmented[ID]

{'ADDENDUM': {'ADDENDUM': 'ADDENDUM  08/31/2017 09:42AM ***Some of the lesions measure: *  Lesion in the left frontal lobe measures approximately 19.0 x 13.9mm.*  Lesion in the right frontoparietal junction measures 22.3 x 16.9mm.*  Right inferior frontal lobe lesion measures 8.3 x 6.0 mm.*  Left occipital lobe lesion measures 10.2 x 11.5 mm.*** END OF  ADDENDUM'}, 'Region': {'IMPRESSION': 'IMPRESSION :***    ***Redemonstration of numerous intracranial lesions compatible with metastasis. Interval decrease enhancement and decrease vasogenic edema associated with the numerous supratentorial lesions suggestive of positive treatment response.  ', 'Electronically_Approved': 'Electronically_Approved  By AMMAR CHAUDHRY, M.D./PSCB  TECH: SMNdd: 08-23-2017 10:59ds: 08-31-2017 09:42STATUS: FINAL', 'FINDINGS': 'FINDINGS :Redemonstration of multiple heterogeneously enhancing intracranial lesions with significant interval decrease in associated FLAIR hyperintensity as well as decrease in mass effec

In [7]:
# BrainMets_MRN_report_BrainMRI.loc[BrainMets_MRN_report_BrainMRI.index[map(lambda x: x in need_check_addendum, BrainMets_MRN_report_BrainMRI['ResultSrcNumID'])],'ADDENDUM_0_1']
# map(lambda x: str(x) in need_check_addendum, BrainMets_MRN_report_BrainMRI['ResultSrcNumID'])
addendum_index= BrainMets_MRN_report_BrainMRI.index[map(lambda x: str(x) in need_check_addendum, BrainMets_MRN_report_BrainMRI['ResultSrcNumID'])]
BrainMets_MRN_report_BrainMRI.loc[:,'ADDENDUM_0_1']=0
BrainMets_MRN_report_BrainMRI.loc[addendum_index,'ADDENDUM_0_1']=1
BrainMets_MRN_report_BrainMRI.head()

,MRN,Resourse,TestItemDescr,ResultDatm,ResultSrcNumID,RenderedText,ADDENDUM_0_1
4,997665,Allscripts,MRI Brain wwo Contrast,2014-12-29 16:22:00.000,217200406,IMPRESSION: 1. Bilateral occipital lobe metast...,0
5,997665,Allscripts,MRI Head without + with Contrast,2012-05-24 12:41:00.000,56386323,IMPRESSION:1. Normal MRI of the brain.. HISTO...,0
6,997665,Allscripts,MRI Brain/Cervical Spine without + with Contrast,2012-08-09 18:56:00.000,59724202,IMPRESSION:1.Normal brain2. No meningeal metas...,0
9,997665,Allscripts,MRI Brain wwo Contrast - Rad Onc,2015-01-13 14:25:00.000,221544151,IMPRESSION: 1. Bilateral deep occipital lobe...,0
10,997665,Allscripts,MRI Brain wwo Contrast - Rad Onc,2015-03-24 13:14:00.000,239590878,IMPRESSION: 1. Compared to prior MRI brain Jan...,0


In [20]:
ResultSrcNumID_Segmented_HaveImpression=[i for i in list(set(BrainMets_MRN_report_BrainMRI['ResultSrcNumID'])-set(need_check_1)) if 'IMPRESSION' in BrainMets_MRN_report_BrainMRI_TextSegmented[i]['Region'].keys()]

In [21]:
ResultSrcNumID_Segmented_HaveImpression[0:10]

['160761981',
 '164610328',
 '70013822',
 '54589595',
 '429776245',
 '171508759',
 '70288331',
 '429004563',
 '303830974',
 '60833230']

In [79]:

# too time comsuming
import re
BrainMets_MRN_report_BrainMRI.loc[:,'Metastasis_state']='unknown'
patten_no_meta=['.*?No .*?meta.*?','.*?No .*?abnor.*?','.*?Negative .*?meta.*?','.*?Negative .*?brain.*?','.*?normal .*?brain.*?','.*?without .*?meta.*?']
patterns_no_meta_use=[re.compile(i,re.IGNORECASE) for i in patten_no_meta]
except_patten=['recurrent','new','other','change']
except_patten_use=[re.compile(i,re.IGNORECASE) for i in except_patten]
def NoMeta_Sentence(sentence):
    return any(j.match(sentence)!=None for j in patterns_no_meta_use) and all(j.match(sentence)==None for j in except_patten_use)

for i in list(set(BrainMets_MRN_report_BrainMRI['ResultSrcNumID'])-set(need_check_1)):
    Impression_splited=BrainMets_MRN_report_BrainMRI_TextSegmented[i]['Region']['IMPRESSION'].split('.')
    for sentence in Impression_splited:
        if any(NoMeta_Sentence(x) for x in Impression_splited):
            BrainMets_MRN_report_BrainMRI.loc[BrainMets_MRN_report_BrainMRI['ResultSrcNumID']==i,'Metastasis_state']=0
        else:
            BrainMets_MRN_report_BrainMRI.loc[BrainMets_MRN_report_BrainMRI['ResultSrcNumID']==i,'Metastasis_state']=1
        

In [ ]:
any(NoMeta_Sentence(x) for x in Impression_splited)

# Parallel 
## core=60

In [85]:
meta=re.compile('.*?metas.*?',re.IGNORECASE)
def NoMeta_Sentence(sentence):
    return any(j.match(sentence)!=None for j in patterns_no_meta_use) and all(j.match(sentence)==None for j in except_patten_use)

def Meta_Sentence(sentence):
    return any(j.match(sentence)!=None for j in patterns_no_meta_use) and all(j.match(sentence)!=None for j in except_patten_use)


def MetsCheck_fun(x):
    Impression_splited=BrainMets_MRN_report_BrainMRI_TextSegmented[x]['Region']['IMPRESSION'].split('.')
    if any(Meta_Sentence(x) for x in Impression_splited): # if there is any Meta sentence the report is meta
        return (x, 1)
    else:
        if any(NoMeta_Sentence(x) for x in Impression_splited):# else is they contain any NoMeta sentence the report is no meta
            return(x,0)
        else:
            if meta.match(BrainMets_MRN_report_BrainMRI_TextSegmented[x]['Region']['IMPRESSION'])!=None:
                return (x, 1)
            else:
                return (x, 'not_sure')
from multiprocessing import Pool
def MetsCheck_par(keys):
    return list(Pool(processes=60).imap(MetsCheck_fun,keys))
MetsCheck_result=MetsCheck_par(ResultSrcNumID_Segmented_HaveImpression)


In [86]:
for ResultSrcNumID,result in MetsCheck_result:
    BrainMets_MRN_report_BrainMRI.loc[BrainMets_MRN_report_BrainMRI['ResultSrcNumID']==ResultSrcNumID,'Metastasis_state']=result

In [62]:
ResultSrcNumID_no_impression=list(set(BrainMets_MRN_report_BrainMRI['ResultSrcNumID'])-set(need_check_1)-set(ResultSrcNumID_Segmented_HaveImpression))
No_Impression_index= BrainMets_MRN_report_BrainMRI.index[map(lambda x: str(x) in ResultSrcNumID_no_impression, BrainMets_MRN_report_BrainMRI['ResultSrcNumID'])]
BrainMets_MRN_report_BrainMRI.loc[:,'No_Impression']=0
BrainMets_MRN_report_BrainMRI.loc[No_Impression_index,'No_Impression']='No_Impression'
NeedCheck_index= BrainMets_MRN_report_BrainMRI.index[map(lambda x: str(x) in need_check_1, BrainMets_MRN_report_BrainMRI['ResultSrcNumID'])]
BrainMets_MRN_report_BrainMRI.loc[NeedCheck_index,'No_Impression']='Need_check'
# BrainMets_MRN_report_BrainMRI=BrainMets_MRN_report_BrainMRI.drop(labels='NoImpression',axis=1)

In [87]:
BrainMets_MRN_report_BrainMRI.head()

,MRN,Resourse,TestItemDescr,ResultDatm,ResultSrcNumID,RenderedText,ADDENDUM_0_1,Metastasis_state,No_Impression
4,997665,Allscripts,MRI Brain wwo Contrast,2014-12-29 16:22:00.000,217200406,IMPRESSION: 1. Bilateral occipital lobe metast...,0,1,0
5,997665,Allscripts,MRI Head without + with Contrast,2012-05-24 12:41:00.000,56386323,IMPRESSION:1. Normal MRI of the brain.. HISTO...,0,0,0
6,997665,Allscripts,MRI Brain/Cervical Spine without + with Contrast,2012-08-09 18:56:00.000,59724202,IMPRESSION:1.Normal brain2. No meningeal metas...,0,0,0
9,997665,Allscripts,MRI Brain wwo Contrast - Rad Onc,2015-01-13 14:25:00.000,221544151,IMPRESSION: 1. Bilateral deep occipital lobe...,0,0,0
10,997665,Allscripts,MRI Brain wwo Contrast - Rad Onc,2015-03-24 13:14:00.000,239590878,IMPRESSION: 1. Compared to prior MRI brain Jan...,0,0,0


In [90]:
BrainMets_MRN_report_BrainMRI['Metastasis_state'].value_counts()

0           1370
1           1274
not_sure     974
unknown       73
Name: Metastasis_state, dtype: int64

# Check

In [35]:
ResultSrcNumID_no_impression

['189405514']

In [37]:
BrainMets_MRN_report_BrainMRI_TextSegmented['189405514']

{'Region': {'Electronically_Approved': 'Electronically_Approved  By MARC AWOBULUYI, M.D./NHAWK TECH: SMNdd: 09-05-2014 20:38ds: 09-05-2014 20:38STATUS: FINAL1490289^RI^DC'}}

In [40]:
list(BrainMets_MRN_report_BrainMRI[BrainMets_MRN_report_BrainMRI['ResultSrcNumID']=='189405514']['RenderedText'])

['ORY: PROSTATA CA ALTERED MENTAL STATUS.MRI brain. Comparison: None Clinical history: Prostate cancer. Technique: Multiplanar multi-sequence MRI of the brain was performed pre-and  post contrast. Findings: Extensive marrow infiltrative process along the left posterior frontal/parietal  calvarium with a bilobed heterogeneously enhancing mass that extends into the  extra-axial space overlying the left posterior frontal/ parietal lobe, exerting  mass effect upon the subjacent brain parenchyma measuring up to 11.5 x 1.6 cm  size. Components of hemorrhage suggested on the T1 images. T1 sequence hyperintensity enlargement of the marrow space also seen overlying  the right frontal and parietal calvarium with components of enhancement.  Bilateral dural enhancement. Modest white matter increased T2 signal seen within the left parietal and occipital lobes presumed subjacent edema. Local sulcal effacement. Midline shift, left to right of approximately 8.2 mm.  Nonspecific patchy scattered areas 

# Known Problem(BUG)

In [89]:
for ResultSrcNumID,result in MetsCheck_result:
    if ResultSrcNumID=='221544151':
        print result
# output is no meta BUT

0


In [67]:
BrainMets_MRN_report_BrainMRI_TextSegmented['221544151']['Region']['IMPRESSION'].split('.')
# matched pattern: '.*?normal .*?brain.*?'
### FLAIR images Normal signal void is seen within the vasculature at the base of the brain',

['IMPRESSION : 1',
 '   Bilateral deep occipital lobe heterogeneously enhancing metastases again noted',
 ' No new enhancing lesions in the brain since last MRI 12/29/2014',
 '2',
 '  Increasing peritumoral edema and brain swelling around the larger of the 2 m No midline shift is seen The ventricles and basal cisterns are normal in size and configuration',
 '   There is no evidence of mass, mass effect, midline shift,  hemorrhage or extra-axial fluid collection',
 '  The diffusion weighted images show no evidence of acute ischemia',
 ' There is normal gray-white matter differentiation',
 '  The white matter away from the lesions described above shows normal signal intensity on T2 and FLAIR images Normal signal void is seen within the vasculature at the base of the brain',
 ' The  bony calvarium, paranasal sinuses and orbits are normal',
 '     The craniocervical junction is normal in position',
 ' ] ']

# Save File

In [ ]:
import pickle
def save_file(folder_file_name,var):
    with open(folder_file_name, 'wb') as file2save:
        pickle.dump(var, file2save)
        
def load_file(file_name):
    with open(file_name, 'rb') as file2load:
        return pickle.load(file2load)

In [91]:
BrainMets_MRN_use_dict={}
BrainMets_MRN_use_dict['dataframe']=BrainMets_MRN_report_BrainMRI
BrainMets_MRN_use_dict['segmented_report']=BrainMets_MRN_report_BrainMRI_TextSegmented
BrainMets_MRN_use_dict['need_check']=need_check_1
BrainMets_MRN_use_dict['need_check_addendum']=need_check_addendum

save_file('/home3/data/LUNG_CANCER/sql/NLP_data/cleaned_data/radiology_report/BrainMets_MRN_use_dict',BrainMets_MRN_use_dict)

In [27]:
# BrainMets_MRN_use_dict=load_file('/home3/data/LUNG_CANCER/sql/NLP_data/cleaned_data/radiology_report/BrainMets_MRN_use_dict')

In [92]:
BrainMets_MRN_report_BrainMRI.to_excel('/home3/data/LUNG_CANCER/sql/NLP_data/cleaned_data/radiology_report/BrainMets_MRN_report_BrainMRI.xlsx')

# try

In [47]:
report2region['10941519']['Region']['IMPRESSION'].split('.')

['IMPRESSION :  1',
 ' Apparent chronic pleural thickening or effusion in the left lateral lower hemithorax',
 '2',
 ' Minimal degenerative change at scattered levels of the anterior thoracic spine',
 '3',
 ' No evidence of acute cardiopulmonary disease',
 '  ']

In [201]:
# import re
# patten_no_meta=['.*?(No .*?meta).*?','.*?(No .*?abnor).*?','.*?(Negative .*?meta).*?','.*?(Negative .*?brain).*?','.*?(normal .*?brain).*?','.*?(without .*?meta).*?']
# patterns_no_meta_use=[re.compile(i,re.IGNORECASE) for i in patten_no_meta]
# a=patterns_no_meta_use[0]
b=a.match('no aaa metassss')
b.string,b.start(),b.end()

('no aaa metassss', 0, 11)

In [57]:
a1=re.compile('.*?No .*?meta.*?',re.IGNORECASE)
b1=a1.match('no aaa metassss')
b1.string,b1.start(),b1.end(),'no aaa metassss'[b1.start():b1.end()],a1.match('aaa metassss')


('no aaa metassss', 0, 11, 'no aaa meta', None)

In [85]:
b=a.match('no aaa metassss')
b1=a1.match('no aaa metassss')

AttributeError: 'list' object has no attribute 'match'

In [83]:
b.string,b.end(),b.start()

('no aaa metassss', 11, 0)

In [69]:
a.match('aaa')==None


True

In [ ]:
for i in report2region['10941519']['Region']['IMPRESSION'].split('.'):
    for pattern in patterns_no_meta_use:
        